In [1]:
from PIL import Image
from os import listdir
import imageio as iio
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

datapath = './test_dataset/center_crop'
y_path = 'label.txt'
X_path = [f for f in listdir(datapath) if f.endswith('jpg')]
y=[]
X=[]

In [1]:
#target 레이블 저장
with open(f'{datapath}/{y_path}','r') as label_txt:
    y=list(map(lambda x: int(x.split()[0]),label_txt.readlines()))
    le = LabelEncoder()
    y = le.fit_transform(y)
    label_txt.close()
for i in range(len(X_path)):
    img_path = X_path[i]
    img_data=[]
    img = Image.open(f'{datapath}/{img_path}')
    ih, iw = img.size
    pix = img.load()
    for py in range(100):
        temp = []
        for px in range(100):
            if py <ih and px < iw :
                pixel = pix[py,px]
                temp.append([pixel[0]/255,pixel[1]/255,pixel[2]/255])
            else :
                temp.append((0,0,0))
        img_data.append(temp)
    X.append(img_data[:])
print(f'{len(X)} datas')

1527 datas


In [2]:
X = np.asarray(X)
y= np.asarray(y)
train_images, test_images, train_labels, test_labels = train_test_split(X,y, test_size=0.2,random_state=42)


In [3]:
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score,f1_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold, KFold
from keras.layers import LeakyReLU
score_acc = make_scorer(accuracy_score)
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')

C:\Users\leeyu\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [4]:

def nn_func(activation
            ,optimizer, learning_rate, epochs,
            layers1, layers2, layers3, layers4, dropout_rate,dropout_prob,pooling_size,

            ):
    # optimizer = 1
    # learning_rate = 0.01
    # epochs = 10
    # layers1 = 6
    # layers2 = 5
    # layers3 = 6
    # layers4 = 6
    # dropout_prob = 0.2
    # dropout_rate = 0.3
    # pooling_size = 2



    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                 'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                 'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),
                 'Nadam':Nadam(learning_rate=learning_rate), 'Ftrl':Ftrl(learning_rate=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    layers3 = round(layers3)
    layers4 = round(layers4)
    poolingSize = round(pooling_size)
    def cl_model():
      model = models.Sequential()
      model.add(Conv2D(100, (3, 3), activation=activation, input_shape=(100, 100,3)))
      model.add(MaxPooling2D(pool_size=(poolingSize, poolingSize)))
      model.add(Conv2D(layers1, (3, 3), activation=activation))
      model.add(MaxPooling2D(pool_size=(poolingSize, poolingSize)))
      model.add(Conv2D(layers2, (3, 3), activation=activation))
      model.add(Flatten())
      model.add(Dense(layers3, activation=activation))
      if dropout_prob > 0.5:
        model.add(Dropout(dropout_rate, seed=123))
      model.add(layers.Dense(layers4, activation=activation))
      model.add(Dense(10, activation='softmax'))
      model.compile(optimizer=optimizer,
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])
      return model
    model = cl_model()
    model.fit(train_images,train_labels,epochs=epochs,verbose=None)
    y_predict = np.argmax(model.predict(test_images, verbose=None),axis=1)
    # return f1_score(test_labels, np.argmax(y_predict,axis=1),average=None).mean()
    return accuracy_score(test_labels, y_predict)




In [6]:
params_nn ={
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'epochs':(20,100),
    'dropout_prob':(0,1),
    'dropout_rate':(0,0.3),
    'layers1':(10, 100),
    'layers2':(10, 100),
    'layers3':(10, 100),
    'layers4':(10, 100),
    'pooling_size':(2, 5)

}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_func, params_nn, random_state=111)
nn_bo.maximize(init_points=24, n_iter=4)

|   iter    |  target   | activa... | dropou... | dropou... |  epochs   |  layers1  |  layers2  |  layers3  |  layers4  | learni... | optimizer | poolin... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.3366    | 5.51      | 0.1691    | 0.1308    | 81.54     | 36.58     | 23.42     | 12.02     | 47.82     | 0.2463    | 2.364     | 4.972     |
| 2         | 0.3366    | 2.14      | 0.08119   | 0.2009    | 69.7      | 34.68     | 51.96     | 20.65     | 16.66     | 0.9018    | 5.558     | 4.522     |


KeyboardInterrupt: 

In [ ]:
params_nn = nn_bo.max['params']
learning_rate = params_nn['learning_rate']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
params_nn['activation'] = activationL[round(params_nn['activation'])]
params_nn['epochs'] = round(params_nn['epochs'])
params_nn['layers1'] = round(params_nn['layers1'])
params_nn['pooling_size'] = round(params_nn['pooling_size'])
params_nn['layers2'] = round(params_nn['layers2'])
params_nn['layers3'] = round(params_nn['layers3'])
params_nn['layers4'] = round(params_nn['layers4'])
optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','Adam']
optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
             'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
             'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),
             'Nadam':Nadam(learning_rate=learning_rate), 'Ftrl':Ftrl(learning_rate=learning_rate)}
params_nn['optimizer'] = optimizerD[optimizerL[round(params_nn['optimizer'])]]
print(params_nn)

{'activation': 'relu', 'dropout_prob': 0.21355594928740118, 'dropout_rate': 0.0893324107995704, 'epochs': 79, 'layers1': 11, 'layers2': 13, 'layers3': 76, 'layers4': 55, 'learning_rate': 0.225886505284512, 'optimizer': <keras.src.optimizers.adam.Adam object at 0x000001291C494710>, 'pooling_size': 5}


In [4]:
#이전에 돌려서 얻은 하이퍼파라미터들
params_nn={}
params_nn['activation'] = 'relu'
params_nn['epochs'] = 79
params_nn['layers1'] = 11
params_nn['layers2'] = 13
params_nn['layers3'] = 76
params_nn['layers4'] = 55
params_nn['learning_rate'] = 0.2259
params_nn['optimizer'] = Adam(learning_rate=params_nn['learning_rate'])
params_nn['pooling_size'] = 5
params_nn['dropout_prob'] = 0.2136
params_nn['dropout_rate'] = 0.08933


In [5]:
def optimized_model():
      model = models.Sequential()
      model.add(Conv2D(100, (3, 3), activation=params_nn['activation'], input_shape=(100, 100,3)))
      model.add(MaxPooling2D(pool_size=(params_nn['pooling_size'], params_nn['pooling_size'])))
      model.add(Conv2D(params_nn['layers1'], (3, 3), activation=params_nn['activation']))
      model.add(MaxPooling2D(pool_size=(params_nn['pooling_size'], params_nn['pooling_size'])))
      model.add(Conv2D(params_nn['layers2'], (3, 3), activation=params_nn['activation']))
      model.add(Flatten())
      model.add(Dense(params_nn['layers3'], activation=params_nn['activation']))
      if params_nn['dropout_prob'] > 0.5:
        model.add(Dropout(params_nn['dropout_rate'], seed=123))
      model.add(layers.Dense(params_nn['layers4'], activation=params_nn['activation']))
      model.add(Dense(10, activation='softmax'))
      model.compile(optimizer=params_nn['optimizer'],
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])
      return model

# Center crop

In [6]:
model = optimized_model()
model.fit(train_images,train_labels,epochs=params_nn['epochs'])
y_predict = np.argmax(model.predict(test_images, verbose=None),axis=1)
print(f"""Center-cropped Image
Accuracy : {accuracy_score(test_labels, y_predict)}
F1 Score : {f1_score(test_labels, y_predict, average=None)}
""")


Epoch 1/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 111ms/step - accuracy: 0.3214 - loss: 2062.0833
Epoch 2/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 105ms/step - accuracy: 0.3864 - loss: 1.6983
Epoch 3/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 106ms/step - accuracy: 0.3366 - loss: 1.7551
Epoch 4/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 106ms/step - accuracy: 0.3472 - loss: 1.7114
Epoch 5/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 105ms/step - accuracy: 0.3535 - loss: 1.7532
Epoch 6/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 107ms/step - accuracy: 0.3723 - loss: 1.7183
Epoch 7/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 106ms/step - accuracy: 0.3683 - loss: 1.7352
Epoch 8/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.3727 - loss: 1.7299
Epoch 9/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.3401 - loss: 1.7511
Epoch 10/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 106ms/step - accuracy: 0.3394 - loss: 1.7526
Epoch 11/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 107ms/step - accuracy: 0.3550 - loss: 1.7749
Epoch 12/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 110ms/s

# Padding

In [12]:
datapath = './test_dataset/padding'
X_path = [f for f in listdir(datapath) if f.endswith('jpg')]
padding_y=[]
padding_X=[]
#target 레이블 저장
with open(f'{datapath}/{y_path}','r') as label_txt:
    padding_y=list(map(lambda x: int(x.split()[0]),label_txt.readlines()))
    le = LabelEncoder()
    padding_y = le.fit_transform(padding_y)
    label_txt.close()
for i in range(len(X_path)):
    img_path = X_path[i]
    img_data=[]
    img = Image.open(f'{datapath}/{img_path}')
    ih, iw = img.size
    pix = img.load()
    for py in range(100):
        temp = []
        for px in range(100):
            if py <ih and px < iw :
                pixel = pix[py,px]
                temp.append([pixel[0]/255,pixel[1]/255,pixel[2]/255])
            else :
                temp.append((0,0,0))
        img_data.append(temp)
    padding_X.append(img_data[:])
print(f'{len(padding_X)} data')
padding_X = np.asarray(padding_X)
padding_y = np.asarray(padding_y)

1527 data


In [13]:
p_train_images, p_test_images, p_train_labels, p_test_labels = train_test_split(padding_X,padding_y,test_size = 0.2, random_state=42)

In [17]:
print(len(p_train_images))

1221


In [18]:
model = optimized_model()
model.fit(p_train_images,p_train_labels,epochs=params_nn['epochs'])
y_predict = np.argmax(model.predict(p_test_images, verbose=None),axis=1)
print(f"""Padding Image
Accuracy : {accuracy_score(p_test_labels, y_predict)}
F1 Score : {f1_score(p_test_labels, y_predict, average=None)}
""")


Epoch 1/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 113ms/step - accuracy: 0.3213 - loss: 4055.6614
Epoch 2/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.3634 - loss: 1.7139
Epoch 3/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.3536 - loss: 1.7873
Epoch 4/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.3766 - loss: 1.7092
Epoch 5/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.3508 - loss: 1.7351
Epoch 6/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.3234 - loss: 1.7729
Epoch 7/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.3901 - loss: 1.6834
Epoch 8/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.3675 - loss: 1.7202
Epoch 9/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.3558 - loss: 1.7331
Epoch 10/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.3311 - loss: 1.7265
Epoch 11/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.3358 - loss: 1.7596
Epoch 12/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/s

39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.3729 - loss: 1.7108
Epoch 69/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.3060 - loss: 1.7509
Epoch 70/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.3118 - loss: 1.7672
Epoch 71/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 107ms/step - accuracy: 0.3195 - loss: 1.7467
Epoch 72/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.3406 - loss: 1.7855
Epoch 73/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.3465 - loss: 1.7571
Epoch 74/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.3687 - loss: 1.7062
Epoch 75/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.3719 - loss: 1.7453
Epoch 76/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.3405 - loss: 1.7347
Epoch 77/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.3234 - loss: 1.7397
Epoch 78/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.3273 - loss: 1.7560
Epoch 79/79
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - 